# Wallethub Data Science Challenge

____ 
## Objective:
- Build a model to predict the value of 'y' based on 304 features. 

## Description
- This notebook was used to clean the data, build, train, and test a model. 
- Loading the data
    - The data was read using pandas.
- Data cleansing:
    - Given that droping NAN reduced the dataset to only 23 rows, I removed columns where the number of NANs was higher than 10% the lenght of the data (10000)
    - The remaining NANs were removed by eliminating the rows containing them
    - This resulted in 92810 rows in 265 features
    - Then I removed an extra 4 columns that contained 1 or 2 values as this might not be that informative to the model 
    - I tried at variuos stages to remove outliers from the data, but that resulted in the elimination of several rows and dramatic reduction of the dataset. 
    - Instead I scale te features, one at a time, using min max scaler from sklearn 
- Building the model:
    - The model is a regression neural network built in tensorflor 
    - It is a simple 3 layer model with an input layer, one hidden layer, and an ouput layer 
    - The cleansed data was splitted into two fractions (80& for training and 20% for testing)
- Training:
    - The model was initially trained using 1000 epochs to check for training and validation (20 % of the training data) loss trends. 
    - The results indicate the the loss (measured in MSE) for both training and validation sets does not change dramatically around 100 epochs
    - A second training with an early stop was performed and the result is that 83 epochs is enough for the model
- Testing
    - The model was then tested on 20% of the cleansed data
    - The testing MSE was 922.82
    - The histogram plot shows relatively low variation 
    - 16633 rows show and absolute error greater than 3
    - The R-square of the model is 0.93
- Saving the model
    - The model was finally saved on a file and reloaded and tested with the same results
    
__To run this code you need to open the notebook on jupyter and selecr 'Run all Cells' in the Run menu__

    * To save time, in the cell with the first training hash out the model.fit step (last line in the cell)   

____ 
# Running the model on other dataset
(with labels 'y' column)

    - The run_model.py script contains code to load the model generated here adn run it agains a dataset with the same structure as the one used here including the 'y' column. 
    - All other columns are treated like here
    
    - To run the model execute: 
    
        python run_model.py -i dataset_00_with_header.csv
    
    - It should print the MSE and save a file with the y column and the predictions for each row. Filename predictions.csv
    
    - The script takes betwee 50 secs and 1:10 mins o run all 100000 in the file dataset_00_with_header
    - It was run on a MACBook Pro 2013 with 8GB memory and Mojave a CPU with Python 3. 

____

In [34]:
# Loading the necesary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from time import time

### - Loading the data

In [2]:
df = pd.read_csv(r'dataset_00_with_header.csv', low_memory = False)
df.shape

(100000, 305)

____
### - Data cleansing

In [3]:
df.dropna().shape

(23, 305)

In [4]:
# Create df with number of NaN on each column
na_cols = pd.DataFrame(df.isnull().sum().sort_values())
na_cols.reset_index(drop = False, inplace=True)
na_cols.columns = (['feature','num_NaN'])
na_cols.loc[:,'tenpct'] = np.where(na_cols.num_NaN > 10000, 'Y', 'N')
print('Total number of columns with NANs: {} \nNumber columns with more than 10K NANs: {}'.format(na_cols.shape[0] , na_cols[na_cols.tenpct =='Y'].shape[0]))
display(na_cols.head())

Total number of columns with NANs: 305 
Number columns with more than 10K NANs: 39


,feature,num_NaN,tenpct
0,x001,0,N
1,x178,0,N
2,x179,0,N
3,x180,0,N
4,x181,0,N


In [6]:
df[na_cols[na_cols.num_NaN !=0].feature.unique()].describe().T.sort_values(by='count').head()

,count,mean,std,min,25%,50%,75%,max
x242,6661.0,1.091186,0.502705,0.0,0.98280,1.1699,1.32660,8.5533
x295,13467.0,0.901304,0.433850,0.0,0.65635,0.9406,1.14095,18.3999
x304,18125.0,1.037957,0.330580,0.0,0.97580,1.0486,1.18620,5.1569
x098,19319.0,44.299446,28.482797,0.0,19.00000,43.0000,67.00000,119.0000
x155,20949.0,1.474820,1.203988,1.0,1.00000,1.0000,1.00000,9.0000


In [7]:
df1 = df[df.columns[df.columns.isin(na_cols[na_cols.tenpct=='Y'].feature.unique())==False]].dropna().reset_index(drop = True)
df1.head()

,x001,x005,x006,x007,x008,x009,x010,x011,x012,x013,...,x291,x292,x294,x296,x298,x299,x300,x301,x303,y
0,1540332,8.0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,706
1,823066,4.0,0,2,2,0,0,0,0,0,...,5206,5206,0,5206,1,1,1,0,0,558
2,1089795,96.0,1,0,0,0,1,3,4,1,...,0,0,0,0,0,0,0,0,0,577
3,1147758,258.0,0,0,0,1,2,1,1,1,...,57762,57762,57762,0,1,1,1,0,0,526
4,1229670,34.0,1,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,496


In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92811 entries, 0 to 92810
Columns: 266 entries, x001 to y
dtypes: float64(2), int64(264)
memory usage: 188.4 MB


In [9]:
dic1= {}
dic2= {}
dic_= {}
for column in df1.columns:
    unique_ = df1[column].nunique()
    if unique_ == 1:
        dic1[column] = [unique_]
    if unique_ ==2:
        dic2[column] = [unique_]
    else:
        dic_[column] = [unique_]

In [10]:
dic1

{'x067': [1], 'x094': [1], 'x095': [1], 'x096': [1]}

In [11]:
df2 = df1[df1.columns[df1.columns.isin(dic1.keys())==False]].dropna().reset_index(drop = True)
df2.shape

(92811, 262)

In [12]:
outliers_all = np.array([], dtype='int64')
for column in df2.columns:
    Q1 = np.percentile(df2[column], 25)
    Q3 = np.percentile(df2[column], 75)
    step = (Q3 - Q1)*2
    out_list_feat=list(df2[~((df2[column] >= Q1 - step) & (df2[column] <= Q3 + step))].index)
    outliers_all = np.append(outliers_all, out_list_feat)

from collections import Counter
out_count = Counter(outliers_all)
out_customer =[]
for customer, count in out_count.items():
    if count > 1:
        out_customer.append(customer.astype(int))
outliers  = list(sorted(out_customer))

print("%d outliers found in more than one feature will be remove" % len(outliers))

df2_no = df2.drop(df2.index[outliers])
    
print(df2.shape, df2_no.shape)

88371 outliers found in more than one feature will be remove
(92811, 262) (4440, 262)


In [13]:
from sklearn.preprocessing import MinMaxScaler
df2n= df2.copy()
for column in df2n.columns[df2n.columns !='y']:
    df2n.loc[:, column ] = MinMaxScaler().fit_transform(df2n[column].astype(np.float).values.reshape(-1,1))
df2n

,x001,x005,x006,x007,x008,x009,x010,x011,x012,x013,...,x291,x292,x294,x296,x298,x299,x300,x301,x303,y
0,0.918371,0.009674,1.0,0.000000,0.009259,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,706
1,0.490582,0.004837,0.0,0.045455,0.018519,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001197,0.000342,0.000000,0.001931,1.0,1.0,1.0,0.0,0.000000,558
2,0.649664,0.116082,1.0,0.000000,0.000000,0.000000,0.030303,0.068182,0.090909,0.023810,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,577
3,0.684234,0.311971,0.0,0.000000,0.000000,0.012346,0.060606,0.022727,0.022727,0.023810,...,0.013282,0.003798,0.015522,0.000000,1.0,1.0,1.0,0.0,0.000000,526
4,0.733087,0.041112,1.0,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,496
5,0.770430,0.152358,0.0,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,528
6,0.546002,0.064087,0.0,0.000000,0.009259,0.037037,0.030303,0.090909,0.000000,0.000000,...,0.005756,0.001646,0.000000,0.009282,1.0,1.0,1.0,0.0,0.000000,551
7,0.932629,0.129383,0.0,0.000000,0.018519,0.037037,0.090909,0.045455,0.136364,0.023810,...,0.012211,0.003492,0.000000,0.019692,1.0,1.0,1.0,0.0,0.067601,607
8,0.549360,0.088271,0.0,0.045455,0.037037,0.012346,0.030303,0.000000,0.022727,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,1.0,1.0,0.0,0.000000,653
9,0.815309,0.216445,0.0,0.000000,0.000000,0.049383,0.030303,0.022727,0.000000,0.047619,...,0.007832,0.002240,0.003902,0.007246,1.0,1.0,1.0,0.0,0.000000,589


In [14]:
outliers_all = np.array([], dtype='int64')
for column in df2n.columns:
    Q1 = np.percentile(df2n[column], 25)
    Q3 = np.percentile(df2n[column], 75)
    step = (Q3 - Q1)*2
    out_list_feat=list(df2n[~((df2n[column] >= Q1 - step) & (df2n[column] <= Q3 + step))].index)
    outliers_all = np.append(outliers_all, out_list_feat)

from collections import Counter
out_count = Counter(outliers_all)
out_customer =[]
for customer, count in out_count.items():
    if count > 1:
        out_customer.append(customer.astype(int))
outliers  = list(sorted(out_customer))

print("%d outliers found in more than one feature will be remove" % len(outliers))

df2nno = df2n.drop(df2.index[outliers])
    
print(df2n.shape, df2nno.shape)

88374 outliers found in more than one feature will be remove
(92811, 262) (4437, 262)


In [15]:
df2n.describe().T

,count,mean,std,min,25%,50%,75%,max
x001,92811.0,0.728385,0.162911,0.0,0.584440,0.739883,0.864079,1.0
x005,92811.0,0.216580,0.150815,0.0,0.106409,0.182588,0.298670,1.0
x006,92811.0,0.338171,0.473090,0.0,0.000000,0.000000,1.000000,1.0
x007,92811.0,0.016990,0.032221,0.0,0.000000,0.000000,0.022727,1.0
x008,92811.0,0.013844,0.021626,0.0,0.000000,0.009259,0.018519,1.0
x009,92811.0,0.015853,0.025683,0.0,0.000000,0.012346,0.024691,1.0
x010,92811.0,0.033495,0.053165,0.0,0.000000,0.000000,0.060606,1.0
x011,92811.0,0.045083,0.062839,0.0,0.000000,0.022727,0.068182,1.0
x012,92811.0,0.039308,0.060027,0.0,0.000000,0.022727,0.045455,1.0
x013,92811.0,0.040401,0.063594,0.0,0.000000,0.023810,0.047619,1.0


____
### - Building the model

In [16]:
# Spliting the data into training and test set
# 80% of the data used for training
train_dataset = df2n.sample(frac = 0.8,random_state = 0)
test_dataset  = df2n.drop(train_dataset.index)
train_labels  = train_dataset.pop('y')
test_labels   = test_dataset.pop('y')

____
### - Training

In [19]:
def plot_history(history):
    # this function generated two plots to visualy check for the model perfomance during the training process
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    # MAE plot
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [y]')
    plt.plot(hist['epoch'], hist['mean_absolute_error'],  label =' Train Error')
    plt.plot(hist['epoch'], hist['val_mean_absolute_error'], label = 'Val Error')
    plt.legend()
    # MSE plot
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$y^2$]')
    plt.plot(hist['epoch'], hist['mean_squared_error'], label = 'Train Error')
    plt.plot(hist['epoch'], hist['val_mean_squared_error'], label = 'Val Error')
    plt.legend()
    plt.show()

#plot_history(history)

In [20]:
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')

EPOCHS = 1000
#history = model.fit(train_dataset, train_labels, epochs = EPOCHS, validation_split = 0.2, verbose = 0, callbacks = [PrintDot()])

In [70]:
# Generating the model
def reg_model(data):
    model = keras.Sequential([
                    # input
                    layers.Dense(64, activation = tf.nn.relu, input_shape = [len(data.keys())]),
                    # hidden 
                    layers.Dense(64, activation = tf.nn.relu),
                    # output
                    layers.Dense(1)
                    ])
    # Stting the optimizer and learning curve
#    optimizer = tf.keras.optimizers.Adam(0.001)
    optimizer = tf.keras.optimizers.Adagrad(lr=0.002)
    # compiling the model
    # both mse and mae will be used to evaluate the model
    model.compile(loss = 'mean_squared_error', optimizer = optimizer, metrics = ['mean_absolute_error', 'mean_squared_error'])
    return model

model = reg_model(train_dataset)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_42 (Dense)             (None, 64)                16768     
_________________________________________________________________
dense_43 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_44 (Dense)             (None, 1)                 65        
Total params: 20,993
Trainable params: 20,993
Non-trainable params: 0
_________________________________________________________________


In [71]:
# Re-training to obtain the best model 
# e.g. epoch where the loss in the training and the validation changes the minimum. 
start = time()
model = reg_model(train_dataset)
early_stop = keras.callbacks.EarlyStopping(monitor ='val_loss', patience = 10)
history = model.fit(train_dataset, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose = 0, callbacks = [early_stop, PrintDot()])
end = time()
print('The run lasted {} minutes'.format(((end - start)/60)))
# Plot the figures
plot_history(history)


....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
..........

KeyboardInterrupt: 

In [ ]:
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=0)
print("Testing set Mean Abs Error: {:5.2f} y".format(mse))

In [ ]:
# Top epoch
max(history.epoch)

____
### - Testing

In [ ]:
test_predictions = model.predict(test_dataset).flatten()
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error")
_ = plt.ylabel("Count")

In [ ]:
# Checking for absolute error differences
error[abs(error) > 3].shape[0]

In [ ]:
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=0)
print("Testing set MSE: {:5.2f} y".format(mse))

In [ ]:
linear_regressor = LinearRegression()
linear_regressor.fit(test_labels.values.flatten().reshape(-1, 1), test_predictions.reshape(-1, 1))
print('R^2 =', linear_regressor.score(test_labels.values.flatten().reshape(-1, 1), test_predictions.reshape(-1, 1)))

__The model needs some adjustments to improve this metric, adjustments could be made by:__
    
    Changing the optimizer or the activation functions
    Adding more layers
    Adding a drop layer after the input layer
    Adjusting the learning curve
____

### - Saving the model 

In [ ]:
model.save('prediction_model.h5')

In [ ]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('prediction_model.h5')
new_model.summary()

In [ ]:
loss, mae, mse = new_model.evaluate(test_dataset, test_labels, verbose=0)
print("Testing set MSE: {:5.2f} y".format(mse))